# Chatbot Implementation with LangChain and LangGraph

This notebook demonstrates the implementation of a conversational chatbot using LangChain and LangGraph. The implementation covers various aspects of chatbot development including:

- Basic model initialization and interaction
- Conversation memory management
- State management with LangGraph
- Custom prompt templates
- Message trimming and token management
- Streaming responses

The notebook serves as a comprehensive guide for building production-ready chatbots with memory persistence and advanced conversation management features.

## 1. Environment Setup

The initial setup involves loading environment variables and configuring the project settings. This section ensures that the necessary API keys and configuration parameters are properly loaded from the `.env` file.

**Key Components:**
- `load_dotenv()`: Loads environment variables from the `.env` file
- `LANGSMITH_PROJECT`: Sets the project identifier for LangSmith integration

**Implementation Notes:**
- Environment variables should be properly configured in the `.env` file
- The project name is set for tracking and monitoring purposes

In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os
  
load_dotenv(override=True)
os.environ["LANGSMITH_PROJECT"]

'llm-playground'

## 2. Model Initialization

This section initializes the language model using LangChain's chat model interface. The implementation demonstrates how to:

- Initialize a chat model with specific parameters
- Configure model provider settings
- Set up the model for conversational interactions


In [2]:
# Initialize the model
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

## 3. Basic Model Interaction

Demonstrates the fundamental interaction with the language model using LangChain's message types. This section shows:

- How to send messages to the model
- How to handle model responses
- The stateless nature of basic model interactions

**Implementation Notes:**
- Uses `HumanMessage` for user inputs
- Demonstrates that the model doesn't maintain conversation state by default
- Shows how to access and process model responses

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BMolTpUEv4D4KMubMUPgOYfDPHBgN', 'finish_reason': 'stop', 'logprobs': None}, id='run-d7dfbf95-cc03-40f8-9001-4e8b0d407af0-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
# The model does not remember previous messages
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. If you'd like to share it or any other information, feel free to do so!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 11, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_80cf447eee', 'id': 'chatcmpl-BMooXtWMQB5wFadFifUFS7qcTI8iL', 'finish_reason': 'stop', 'logprobs': None}, id='run-ffe1518f-cdb8-4be6-9640-13b113ea976f-0', usage_metadata={'input_tokens': 11, 'output_tokens': 49, 'total_tokens': 60, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [5]:
# Passing the conversation history
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?"),
]

model.invoke(messages)

AIMessage(content='Your name is Bob. How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 33, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BMopaN69ka9cXHB2VOukolzFl40jL', 'finish_reason': 'stop', 'logprobs': None}, id='run-c52fccc1-62a1-4136-ab90-aff239528417-0', usage_metadata={'input_tokens': 33, 'output_tokens': 15, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 4. Conversation Memory Management

This section implements conversation memory using LangGraph's in-memory checkpointer. Key features include:

- State management with `MessagesState`
- Thread-based conversation isolation
- Persistent conversation history

**Technical Implementation:**
- Uses `MemorySaver` for state persistence
- Implements thread-based conversation management
- Demonstrates state restoration across multiple interactions

In [7]:
# Use LangGraph in-memory checkpointer
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [8]:
# First query
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! How can I assist you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! How can I help you today?


In [10]:
# Change the thread ID restarts the conversation

config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t have access to personal data about individuals unless it's shared with me in the course of our conversation. So, I don’t know your name. If you’d like to share it or have any other questions, feel free to let me know!


In [11]:
# Use the same thread ID to continue the conversation
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! If there's anything else you'd like to discuss, feel free to let me know.


In [12]:
# Asynchronous execution
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about you unless you've shared it during our conversation. How can I assist you today?


## 5. Custom Prompt Templates

Shows how to implement custom prompt templates for specialized chatbot behaviors. This section covers:

- Creating custom prompt templates
- Implementing role-based responses
- Managing conversation context

**Key Features:**
- Uses `ChatPromptTemplate` for template management
- Implements system messages for behavior control
- Demonstrates variable substitution in prompts

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! What be bringin’ ye to the high seas of conversation today? Speak up, matey!


In [15]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Jim, if I be rememberin’ correctly! What else can I help ye with, me hearty?


In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [17]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [18]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


In [19]:
#Entire state is saved, we can omit language if we don't want to change it

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


## 6. Message Trimming and Token Management

Implements conversation history management with token-based trimming. This section demonstrates:

- Token-based message trimming
- Conversation history optimization
- Memory management strategies

**Technical Details:**
- Uses `trim_messages()` for token management
- Implements configurable trimming strategies
- Shows how to handle partial message preservation

In [46]:
# Handling conversation history

from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=80,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [47]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [37]:
# The model does not remember the name since it was trimmed
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. If you'd like to share it, feel free!


In [48]:
# The model remembers the math problem since it was not trimmed
config = {"configurable": {"thread_id": "abc6791"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked, "What's 2 + 2?"


In [33]:
# Stream the response

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:

|Why| did| the| scare|crow| win| an| award|?

|Because| he| was| outstanding| in| his| field|!||